---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

In [64]:
def blight_model():
    import pandas as pd
    import numpy as np
    from datetime import datetime
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import auc

    def time_gap(hearing_date_str, ticket_issued_date_str):
        from datetime import datetime      
        if not hearing_date_str or type(hearing_date_str)!=str: return 0
        hearing_date = datetime.strptime(hearing_date_str, "%Y-%m-%d %H:%M:%S")
        ticket_issued_date = datetime.strptime(ticket_issued_date_str, "%Y-%m-%d %H:%M:%S")
        gap = hearing_date - ticket_issued_date
        return gap.days

    dfTrain=pd.read_csv('train.csv',encoding = "ISO-8859-1")
    dfTrain=dfTrain.loc[dfTrain['compliance'].isin([0.0, 1.0])]
    dfAddr =  pd.read_csv('addresses.csv')
    dfll = pd.read_csv('latlons.csv')
    dfAddress = dfAddr.set_index('address').join(dfll.set_index('address'), how='left')
    dfTrain = dfTrain.set_index('ticket_id').join(dfAddress.set_index('ticket_id'))
    dfTrain=dfTrain.fillna(method='pad',axis=1)
    dfTrain['time_gap'] = dfTrain.apply(lambda row: time_gap(row['hearing_date'], row['ticket_issued_date']), axis=1)
    dfTrain.loc[dfTrain.time_gap<0, 'time_gap']=0

    list_to_remove_train = [
    'balance_due',
    'collection_status',
    'compliance_detail',
    'payment_amount',
    'payment_date',
    'payment_status'
    ]
    list_to_remove_all = ['violator_name','zip_code', 'country', 'city','state','violation_street_number', 'violation_street_name',
          'violation_zip_code','violation_description','admin_fee','state_fee',
          'mailing_address_str_number', 'mailing_address_str_name',
          'non_us_str_code','ticket_issued_date', 'hearing_date', 'grafitti_status']
    Get_dummy_col=['agency_name', 'inspector_name', 'violation_code','disposition']


    dfTrain.drop(list_to_remove_train,axis=1,inplace=True)
    y_train=dfTrain['compliance']
    dfTrain.drop('compliance',axis=1,inplace=True)
    X_train=dfTrain.drop(list_to_remove_all, axis=1)
    X_train=pd.get_dummies(X_train, columns=Get_dummy_col)

    dfTest=pd.read_csv('test.csv')
    dfTest = dfTest.set_index('ticket_id').join(dfAddress.set_index('ticket_id'))
    dfTest['time_gap'] = dfTest.apply(lambda row: time_gap(row['hearing_date'], row['ticket_issued_date']), axis=1)

    dfTest.drop(list_to_remove_all, axis=1, inplace=True)
    X_test=pd.get_dummies(dfTest, columns=Get_dummy_col)

    train_features = set(X_train)    
    for feature in set(train_features):
        if feature not in X_test:
            train_features.remove(feature)
    train_features = list(train_features)
    X_train = X_train[train_features]
    X_test = X_test[train_features]

    X_train=X_train.fillna(method='pad',axis=1)
    X_test=X_test.fillna(method='pad',axis=1)

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    y_train=y_train.astype('float').astype('int')
    clf = RandomForestClassifier()
    y_proba=clf.fit(X_train_scaled, y_train).predict_proba(X_test_scaled)
    y_proba=y_proba[:,1]
    X_test['compliance']=y_proba
    
    return X_test.compliance




In [63]:
# x=blight_model()
# x

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.1
285362    0.0
285361    0.3
285338    0.1
285346    0.2
285345    0.0
285347    0.1
285342    0.9
285530    0.0
284989    0.0
285344    0.0
285343    0.0
285340    0.0
285341    0.0
285349    0.2
285348    0.0
284991    0.0
285532    0.0
285406    0.0
285001    0.0
285006    0.0
285405    0.0
285337    0.0
285496    0.5
285497    0.1
285378    0.0
285589    0.1
285585    0.1
285501    0.1
285581    0.0
         ... 
376367    0.0
376366    0.0
376362    0.0
376363    0.1
376365    0.0
376364    0.0
376228    0.4
376265    0.0
376286    0.0
376320    0.1
376314    0.0
376327    0.0
376385    0.2
376435    0.7
376370    0.9
376434    0.0
376459    0.0
376478    0.2
376473    0.0
376484    0.1
376482    0.3
376480    0.0
376479    0.0
376481    0.0
376483    0.0
376496    0.0
376497    0.0
376499    0.0
376500    0.0
369851    0.7
Name: compliance, dtype: float64